# Low-Rank Matrix Completion (LRMC)

In [ ]:
import numpy as np

def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

def svt(mat, tau):
    u, s, v = np.linalg.svd(mat, full_matrices = 0)
    vec = s - tau
    vec[vec < 0] = 0
    return u @ np.diag(vec) @ v

def LRMC(dense_tensor, sparse_tensor, rho, maxiter):
    if np.isnan(sparse_tensor).any() == False:
        pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    elif np.isnan(sparse_tensor).any() == True:
        pos_test = np.where((dense_tensor > 0) & (np.isnan(sparse_tensor)))
        sparse_tensor[np.isnan(sparse_tensor)] = 0
    pos_train = np.where(sparse_tensor != 0)
    tensor_test = dense_tensor[pos_test]
    X = sparse_tensor.copy()
    Z = sparse_tensor.copy()
    T = sparse_tensor.copy()
    show_iter = 10
    for it in range(maxiter):
        for t in range(sparse_tensor.shape[2]):
            Z[:, :, t] = svt(X[:, :, t] + T[:, :, t] / rho, 1 / rho)
        X = Z - T / rho
        X[pos_train] = sparse_tensor[pos_train]
        T = T - rho * (Z - X)
        if (it + 1) % show_iter == 0:
            print(it + 1)
            print(compute_mape(tensor_test, X[pos_test]))
            print(compute_rmse(tensor_test, X[pos_test]))
            print()
    return X

## HighD Dataset

Hyperparameters:

- $\rho=10^{-3}$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=3):
    fig = plt.figure(figsize = (2.5*2.5, 5))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower', 
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 5, vmax = 35, fignum = 1)
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/HighD/speed_matrix_full_46.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/HighD/speed_matrix_70_46.npy').transpose(1, 0, 2)

for lane in range(3):
    plot_speed_field(dense_tensor[:, :, lane],
                     'speed_matrix_full_46_lane{}.png'.format(lane + 1))
    plot_speed_field(sparse_tensor[:, :, lane],
                     'speed_matrix_70_46_lane{}.png'.format(lane + 1))

import time
start = time.time()
rho = 1e-3
maxiter = 100
tensor_hat = LRMC(dense_tensor, sparse_tensor, rho, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

for lane in range(3):
    plot_speed_field(tensor_hat[:, :, lane], 
                     'HighD_speed_field_70_LRMC_rec_lane{}.png'.format(lane + 1))

## CitySim Dataset

Hyperparameters:

- $\rho=10^{-3}$


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import imageio as io
plt.rcParams['font.size'] = 12

def plot_speed_field(data, filename, x_scale=2, y_scale=5):
    fig = plt.figure(figsize = (2.3*2.3, 1.8))
    x = range(data.shape[1])
    x_new = [i * x_scale for i in x]
    y = range(data.shape[0])
    y_new = [i * y_scale for i in y]
    plt.matshow(data, cmap='jet_r', origin='lower',
                extent=[x_new[0], x_new[-1], y_new[0], y_new[-1]],
                vmin = 14, vmax = 35, fignum = 1, aspect='auto')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.xlabel('Time (s)')
    plt.ylabel('Location (m)')
    cbar = plt.colorbar(fraction = 0.015)
    cbar.ax.set_ylabel('Speed (m/s)')
    plt.show()
    fig.savefig(filename, bbox_inches = 'tight', dpi = 300)

dense_tensor = np.load('../datasets/CitySim/speed_matrix_full.npy').transpose(1, 0, 2)
sparse_tensor = np.load('../datasets/CitySim/speed_matrix_70_FB.npy').transpose(1, 0, 2)

for lane in range(3):
    plot_speed_field(dense_tensor[:, :, lane],
                     'speed_matrix_full_lane{}.png'.format(lane + 1))
    plot_speed_field(sparse_tensor[:, :, lane],
                     'speed_matrix_70_lane{}.png'.format(lane + 1))

import time
start = time.time()
rho = 1e-3
maxiter = 100
tensor_hat = LRMC(dense_tensor, sparse_tensor, rho, maxiter)
end = time.time()
print('Running time: %d seconds.'%(end - start))

for lane in range(3):
    plot_speed_field(tensor_hat[:, :, lane], 
                     'CitySim_speed_field_70_LRMC_rec_lane{}.png'.format(lane + 1))

### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>